# Acquire ACS Data Over Serial WITH Knowledge From the Device File
For this example, you will need to connect an ACS to your computer via serial. 
The sensor should be on before running this notebook. Note that the ACS is in picoDOS for the first 10 seconds after it is powered on and may not register on a serial port.

An example for working with ACS data WITHOUT knowledge of the device file can be found [here](https://github.com/IanTBlack/acspype/tree/main/examples).
It is strongly recommended that you rename your device file to include the date of calibration. ACS device files arrive from the factory with the format of 'ACS-XXX.dev', making it possible to overwrite old device files if you are not careful.

Note: In this example, absorption and attenuation will look bad because the ACS is running in air and using the pure water offsets.

In [1]:
from acspype import ACSStream, ACSDev
from acspype.processing import convert_sn_str
from acspype.utils import find_acs_port

In [2]:
## Load the Device File and TS-Correction Information
dev = ACSDev('../dev_tools/test_files/ACS-00011_2022-10-20.DEV')

In [3]:
num_packets = 5  # Acquire X packets, and then stop. Otherwise, the while loop will run forever.
dummy_temperature = 12.345  # Filler value for temperature because there is no connected CTD/TSG.
dummy_salinity = 34.567 # Filler value for salinity because there is no connected CTD/TSG.

In [4]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
with ACSStream(port) as acss: # Open an ACS serial object.
    i = 0
    while True:  # Continuously read the stream until a full packet is found.
        acss.read_stream()
        acs_pkt = acss.find_packet()
        if acs_pkt.full_packet is not None: # If a full packet is found, then perform operations on it.
            parsed_packet = acss.parse_packet(acs_pkt) # Parse the packet.
            
            calibrated_packet = acss.calibrate_packet(parsed_packet, dev) # Apply offsets and delta_t values from the device file.
            
            ts_corrected_packet = acss.ts_correct_packet(calibrated_packet, dummy_temperature, dummy_salinity, dev)            
            
            # Print some of the results.
            print(f"---------- Packet {i+1} ----------")
            print(f"Acquisition Time: {calibrated_packet.daq_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'}")
            print(f"Acquired Packet From: {convert_sn_str(parsed_packet.sn_int)}")
            print(f"Remaining # Bytes in Buffer: {len(acss._buffer)}")
            print(f"Absorption Wavelengths: {calibrated_packet.a_wavelength}")
            print(f"Absorption With Discontinuity: {calibrated_packet.a_m_discontinuity}")
            print(f"Absorption Discontinuity Offset: {calibrated_packet.a_discontinuity_offset}")
            print(f"Measured Absorption (a_m): {calibrated_packet.a_m}")
            print(f"Absorption TS-Corrected (a_mts): {ts_corrected_packet.a_mts}")
            print('\n')
            
            i += 1
            if i == num_packets:
                break

---------- Packet 1 ----------
Acquisition Time: 2025-05-07T16:52:28.459Z
Acquired Packet From: ACS-00011
Remaining # Bytes in Buffer: 19
Absorption Wavelengths: [401.8 405.3 408.7 412.1 415.5 419.  423.3 427.6 431.3 435.4 439.1 442.8
 447.4 451.8 455.9 459.9 464.1 468.4 472.6 477.3 482.  486.4 490.6 494.8
 498.8 502.8 506.9 511.8 516.5 521.  524.9 529.2 532.9 537.2 541.3 545.2
 549.7 553.8 558.2 562.5 566.2 570.3 573.7 577.3 581.  584.6 586.9 590.7
 595.  599.6 603.8 608.  612.3 616.9 621.2 625.2 629.4 633.4 637.3 641.6
 646.  650.9 655.2 660.9 665.  668.5 672.9 677.1 681.1 685.5 689.2 692.6
 696.3 700.4 704.4 707.8 711.6 715.3 719.5 723.5 727.3 731.1 735.  738.9]
Absorption With Discontinuity: [ 0.36731799  0.42890932  0.46231029  0.48640753  0.49641034  0.50220171
  0.49773165  0.49441689  0.49582047  0.48592507  0.48591996  0.47968071
  0.47895394  0.47636272  0.47118227  0.47020523  0.46822459  0.46667846
  0.46325554  0.46255096  0.45876025  0.456259    0.45485681  0.45158186
  0